In [0]:
#This Code predict Assignee for Roslyn Issues with Title

import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn import preprocessing
import keras
import numpy as np

url = "https://tfhub.dev/google/universal-sentence-encoder-large/3"
embed = hub.Module(url)

data = pd.read_csv('IssueDetailsRoslyn_20102019.csv', encoding='latin-1')

y = list(data['Assignee']) 
x = list(data['Title_Description']) 

le = preprocessing.LabelEncoder()
le.fit(y)

categories = list(set(y))

def encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def decode(le, one_hot):
    dec = np.argmax(one_hot, axis=1)
    return le.inverse_transform(dec)

test = encode(le,categories)

untest = decode(le, test)


x_enc = x
y_enc = encode(le, y)

x_train = np.asarray(x_enc[:800])
y_train = np.asarray(y_enc[:800])


x_test = np.asarray(x_enc[800:])
y_test = np.asarray(y_enc[800:])

In [0]:
from keras.layers import Input, Lambda, Dense
from keras.models import Model
import keras.backend as K

def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)))

In [63]:
input_text = Input(shape=(1,), dtype=tf.string)
embedding = Lambda(UniversalEmbedding, output_shape=(512, ))(input_text)
dense = Dense(256, activation='relu')(embedding)
pred = Dense(len(categories), activation='softmax')(dense)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [64]:
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())  
    session.run(tf.tables_initializer())
    history = model.fit(x_train, y_train, epochs=30, batch_size=32)
    model.save_weights('./model_roslyn_TitleDescription_Assignee_Prediction.h5')

Epoch 1/30
800/800 [==============================] - 77s 96ms/step - loss: 5.4947 - acc: 0.0762
Epoch 2/30
800/800 [==============================] - 63s 79ms/step - loss: 4.7410 - acc: 0.0812
Epoch 3/30
800/800 [==============================] - 63s 79ms/step - loss: 4.4077 - acc: 0.1475
Epoch 4/30
800/800 [==============================] - 64s 80ms/step - loss: 4.1972 - acc: 0.1662
Epoch 5/30
800/800 [==============================] - 72s 90ms/step - loss: 4.0274 - acc: 0.1787
Epoch 6/30
800/800 [==============================] - 63s 78ms/step - loss: 3.8802 - acc: 0.1988
Epoch 7/30
800/800 [==============================] - 62s 78ms/step - loss: 3.7482 - acc: 0.2225
Epoch 8/30
800/800 [==============================] - 63s 79ms/step - loss: 3.6278 - acc: 0.2112
Epoch 9/30
800/800 [==============================] - 63s 79ms/step - loss: 3.5098 - acc: 0.2263
Epoch 10/30
800/800 [==============================] - 63s 79ms/step - loss: 3.4004 - acc: 0.2438
Epoch 11/30
800/800 [========

In [0]:
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    model.load_weights('./model_roslyn_TitleDescription_Assignee_Prediction.h5')  
    predicts = model.predict(x_test, batch_size=32)  

In [66]:
y_test = np.asarray(y_enc[800:])


y_test = decode(le, y_test)
y_preds = decode(le, predicts)

from sklearn import metrics

metrics.confusion_matrix(y_test, y_preds)

print(metrics.classification_report(y_test, y_preds))


                                                                                                                   precision    recall  f1-score   support

                                                                                                   333fred|gafter       0.00      0.00      0.00         1
                                                                         333fred|jaredpar|heejaechang|tmat|agocke       0.00      0.00      0.00         1
333fred|jasonmalinowski|ryzngard|jaredpar|agocke|gafter|RikkiGibson|VSadov|AlekseyTs|cston|jcouv|chsienki|fayrose       0.00      0.00      0.00         0
                                                                                            333fred|tmat|jaredpar       0.00      0.00      0.00         1
                                                                                                        AlekseyTs       0.00      0.00      0.00         3
                                                                     

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


# New Section